In [100]:
import pandas as pd
import gradio as gr
import torch
import accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import os 
from mistralai import Mistral


In [101]:
API_KEY = "DNN3lk0n8cBucYBe0s38s6gCIw4X6zmP"

In [102]:
USE_MISTRAL = True
client = Mistral(api_key="DNN3lk0n8cBucYBe0s38s6gCIw4X6zmP")

In [103]:
from mistralai import Mistral


client = Mistral(api_key=API_KEY)

models = client.models.list()

print("Available Mistral models:\n")
for model in models.data:
    print("-", model.id)


Available Mistral models:

- mistral-medium-2505
- mistral-large-latest
- mistral-medium-2508
- mistral-medium-latest
- mistral-medium
- ministral-3b-2410
- ministral-3b-latest
- ministral-8b-2410
- ministral-8b-latest
- open-mistral-7b
- mistral-tiny
- mistral-tiny-2312
- open-mistral-nemo
- open-mistral-nemo-2407
- mistral-tiny-2407
- mistral-tiny-latest
- open-mixtral-8x7b
- mistral-small
- mistral-small-2312
- open-mixtral-8x22b
- open-mixtral-8x22b-2404
- mistral-small-2409
- mistral-large-2407
- mistral-large-2411
- pixtral-large-2411
- pixtral-large-latest
- mistral-large-pixtral-2411
- codestral-2501
- codestral-2412
- codestral-2411-rc5
- codestral-2508
- codestral-latest
- devstral-small-2505
- devstral-small-2507
- devstral-small-latest
- devstral-medium-2507
- devstral-medium-latest
- pixtral-12b-2409
- pixtral-12b
- pixtral-12b-latest
- mistral-small-2501
- mistral-small-2503
- mistral-small-2506
- mistral-small-latest
- mistral-saba-2502
- mistral-saba-latest
- magistral-

In [104]:
df = pd.DataFrame([
    {"company": "X", "amount": 1200, "date": "2023-05-10", "details": "Contract for marketing"},
    {"company": "Y", "amount": 500, "date": "2022-11-20", "details": "Contract for supply"},
    {"company": "X", "amount": 800, "date": "2023-01-15", "details": "Contract for consulting"},
    {"company": "Z", "amount": 2500, "date": "2023-06-01", "details": "IT infrastructure project"},
    {"company": "A", "amount": 1500, "date": "2021-12-30", "details": "Office renovation contract"},
    {"company": "B", "amount": 3000, "date": "2023-03-22", "details": "Marketing campaign"},
    {"company": "C", "amount": 750, "date": "2022-08-10", "details": "Supply of furniture"},
    {"company": "X", "amount": 2000, "date": "2023-07-15", "details": "Consulting project"},
    {"company": "Y", "amount": 1800, "date": "2023-01-05", "details": "Software license"},
    {"company": "Z", "amount": 400, "date": "2022-05-17", "details": "Maintenance contract"},
    {"company": "A", "amount": 2200, "date": "2023-04-10", "details": "Training program"},
    {"company": "B", "amount": 950, "date": "2021-11-01", "details": "Marketing research"},
    {"company": "C", "amount": 1300, "date": "2023-02-25", "details": "Office furniture supply"},
    {"company": "X", "amount": 5000, "date": "2023-08-01", "details": "Major consulting project"},
    {"company": "Y", "amount": 1250, "date": "2022-09-30", "details": "Software maintenance"},
])


In [105]:
file_path = "document_metadata.csv"

df = pd.read_csv(file_path)

In [106]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM

# model_path = "./bloomz-1b1-local"

# tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
# device = torch.device("cpu")



In [107]:
# save_path = "./flan-t5-local"
# tokenizer.save_pretrained(save_path)
# model.save_pretrained(save_path)

In [108]:
import re, json
def clean_json_string(s: str) -> str:
    s = s.strip()

    if not s.startswith("{") and "company" in s:
        s = "{" + s
    if not s.endswith("}"):
        s = s + "}"

    s = re.sub(r",\s*}", "}", s)
    s = re.sub(r"\"\s*}", "}", s)
    return s


In [109]:
import json
from mistralai import Mistral

USE_MISTRAL = True 

if USE_MISTRAL:
    client = Mistral(api_key=API_KEY)


def query_to_filters(query: str, max_new_tokens: int = 150):
    """
    Convert English or Arabic query about contracts into a structured filter dict:
    { company, amount_min, amount_max, year_min, year_max, keywords }
    """

    prompt = f"""
Convert this query about documents into a JSON object with keys:
author, category, year_min, year_max, tags, keywords.
Use null if not specified. Respond ONLY with valid JSON.
- The 'keywords' field should always be in English (the language of the CSV),
even if the input query is in Arabic.
- If multiple tags are mentioned, return them as a list.
- If a date range is mentioned, fill year_min and year_max.
- If only a single year is mentioned, set both year_min and year_max to that year.
- If a field is not mentioned, set it to null.

English examples:
"Reports by John Smith" => {{"author":"John Smith","category":"Report","year_min":null,"year_max":null,"tags":null,"keywords":null}}
"Documents about financial performance in 2023" => {{"author":null,"category":null,"year_min":2023,"year_max":2023,"tags":null,"keywords":"financial performance"}}
"Policies by HR with tag onboarding between 2021 and 2022" => {{"author":"HR","category":"Policy","year_min":2021,"year_max":2022,"tags":["onboarding"],"keywords":null}}
"Documents tagged marketing and sales in 2023" => {{"author":null,"category":null,"year_min":2023,"year_max":2023,"tags":["marketing","sales"],"keywords":null}}

Arabic examples:
"تقارير من جون سميث" => {{"author":"John Smith","category":"Report","year_min":null,"year_max":null,"tags":null,"keywords":null}}
"مستندات عن الأداء المالي في ٢٠٢٣" => {{"author":null,"category":null,"year_min":2023,"year_max":2023,"tags":null,"keywords":"financial performance"}}
"سياسات من قسم الموارد البشرية مع علامات onboarding بين 2021 و 2022" => {{"author":"HR","category":"Policy","year_min":2021,"year_max":2022,"tags":["onboarding"],"keywords":null}}
"مستندات عن التسويق والمبيعات في 2023" => {{"author":null,"category":null,"year_min":2023,"year_max":2023,"tags":["marketing","sales"],"keywords":null}}

Query: "{query}"
JSON:
"""


    if USE_MISTRAL:
        response = client.chat.complete(
            model="ministral-3b-latest",  
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        raw_output = response.choices[0].message.content

    try:
        start = raw_output.find("{")
        end = raw_output.rfind("}") + 1
        json_str = raw_output[start:end]
        filters = json.loads(json_str)
        
        for key in [ "document_id","title","author","created_date","last_modified","category","tags","content"]:
            if key not in filters:
                filters[key] = None

    except Exception as e:
        print("!!! Fallback triggered: returning empty filters")
        print("Raw model response:", raw_output)
        filters = {
            "author": None,
            "category": None,
            "year_min": None,
            "year_max": None,
            "tags": None,
            "keywords": None
        }

    return filters


In [110]:
import arabic_reshaper
from bidi.algorithm import get_display
import re

def normalize_arabic(text):
    if not isinstance(text, str):
        return ""
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا").replace("ى", "ي").strip()
    reshaped_text = arabic_reshaper.reshape(text)
    bidi_text = get_display(reshaped_text)
    return bidi_text


In [131]:
def compute_embeddings(df, embed_client):
    embeddings = []
    for i, row in df.iterrows():
        text = f"{row['title']} {row['tags']} {row['content']}"
        response = embed_client.embeddings.create(
            model="mistral-embed",
            inputs=[text]
        )
        embeddings.append(response.data[0].embedding)
    df["embedding"] = embeddings
    return df


In [128]:
from mistralai import Mistral
import os

with Mistral(api_key=API_KEY) as mistral:
    response = mistral.embeddings.create(
        model="mistral-embed",
        inputs=["Hello world, this is a test for embeddings."]
    )

embedding_vector = response.data[0].embedding
print(f"Embedding length: {len(embedding_vector)}")


Embedding length: 1024


In [132]:
df = compute_embeddings(df, client)

In [140]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def semantic_search(query, df, top_k=5):
    client = Mistral(api_key=API_KEY)
    response = client.embeddings.create(
        model="mistral-embed",
        inputs=[query] 
    )
    query_vec = np.array(response.data[0].embedding).reshape(1, -1)
    print(type(df))
    print(df.columns)
    print(df.head())

    df_valid = df[df["embedding"].apply(lambda x: isinstance(x, (list, np.ndarray)))].copy()
    doc_embeddings = np.vstack(df_valid["embedding"].apply(lambda x: np.array(x)).to_numpy())

    sims = cosine_similarity(query_vec, doc_embeddings)[0]

    df_valid["similarity"] = sims
    results = df_valid.sort_values("similarity", ascending=False).head(top_k)
    return results[["title", "author", "category", "tags", "similarity"]]



In [142]:
def semantic_search_wrapper(query):
    return semantic_search(query, df)

In [134]:
def search_csv(query):
    filters = query_to_filters(query)
    print("Structured query:", filters)

    results = df.copy()

    if filters.get("author"):
        results = results[results["author"].str.contains(filters["author"], case=False, na=False)]
    if filters.get("category"):
        results = results[results["category"].str.contains(filters["category"], case=False, na=False)]
    if filters.get("year_min"):
        results = results[pd.to_datetime(results["created_date"]).dt.year >= filters["year_min"]]
    if filters.get("year_max"):
        results = results[pd.to_datetime(results["created_date"]).dt.year <= filters["year_max"]]
    if filters.get("tags"):
        tag_pattern = "|".join(filters["tags"])
        results = results[results["tags"].str.contains(tag_pattern, case=False, na=False)]
    if filters.get("keywords"):
        keyword_text = filters["keywords"].strip()
        if keyword_text:
            words = re.split(r"\s+", keyword_text)
            words_normalized = [normalize_arabic(w) for w in words]

            results["content_normalized"] = results["content"].apply(normalize_arabic)

            pattern = "|".join(re.escape(w) for w in words_normalized)
            results = results[results["content_normalized"].str.contains(pattern, case=False, regex=True)]


    return results


In [ ]:
with gr.Blocks() as iface:
    gr.Markdown("# Arabic Contract Search")
    gr.Markdown("Choose structured or semantic search")
    
    with gr.Tab("Structured Search"):
        query_input = gr.Textbox(label="Enter your Arabic query")
        output_table = gr.Dataframe(label="Results")
        search_btn = gr.Button("Search (Structured)")
        search_btn.click(search_csv, inputs=query_input, outputs=output_table)

    with gr.Tab("Semantic Search"):
        sem_query_input = gr.Textbox(label="Enter your query for semantic search")
        sem_output_table = gr.Dataframe(label="Results")
        sem_search_btn = gr.Button("Search (Semantic)")
        sem_search_btn.click(semantic_search_wrapper, inputs=sem_query_input, outputs=sem_output_table)

iface.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


<class 'pandas.core.frame.DataFrame'>
Index(['document_id', 'title', 'author', 'created_date', 'last_modified',
       'category', 'tags', 'content', 'embedding'],
      dtype='object')
  document_id                         title          author created_date  \
0      DOC001  Annual Financial Report 2023      John Smith   2023-01-15   
1      DOC002             Employee Handbook   HR Department   2021-07-01   
2      DOC003          AI Research Proposal        Test Guy   2023-05-05   
3      DOC004    Marketing Strategy Q2 2023       Sarah Lee   2023-04-01   
4      DOC005     Office Renovation Invoice  BuildCorp Ltd.   2022-11-18   

  last_modified  category                         tags  \
0    2023-02-10    Report        finance, annual, 2023   
1    2023-03-20    Policy        hr, onboarding, rules   
2    2023-05-12  Proposal            ai, research, nlp   
3    2023-04-15  Strategy         marketing, q2, sales   
4    2022-11-18   Invoice  invoice, renovation, office   

        